In [2]:
import numpy as np
import matplotlib.pyplot as plt
from numpy import log as ln
import random
from cupy import random
import math
from matplotlib.pyplot import plot,savefig
import sys
filePath = 'Utils/nup'
sys.path.append(filePath)
import functions
import os
from functions import relu,drelu,softmax,divi_,mini_batch_generate,sigmoid,dsigmoid,turn_2_zero,scale,tanh,dtanh
from optimizers import Adam
import model_save
import load_MNIST
mnist=(load_MNIST.load_mnist(one_hot=True))
train_data = mnist[0][0][0:60000].T
train_label = mnist[0][1][0:60000].T
test_data = mnist[1][0][0:10000].T
test_label = mnist[1][1][0:10000].T
print(np.shape(train_data))
print(np.shape(train_label))

(784, 60000)
(10, 60000)


In [15]:
class BPNetwork:
    def __init__(self,sizes):
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.messes=[((1/np.sqrt(self.sizes[l]))*(np.random.normal(0.0,1.0,(self.sizes[l+1],self.sizes[l])))) for l in range(self.num_layers-1)]
        self.Adam_theta = Adam(self.messes)
    
    
    def w_feedforward(self,a,activate,back=False):
        process=[]
        flag=0
        zm=[]
        process=[a]
        for mess in self.messes:
            
            flag=flag+1
            z=(np.dot(mess,a))
            
            if (flag<(self.num_layers-1)):
                a = activate(z)
            if (flag>=(self.num_layers-1)):
                a = softmax(z)
            zm.append(z)
            process.append(a)
        if back == False:
            return process[-1]
        if back == True:
            return process,zm
    
    def evaluate(self, testdata,testlabel,activate):
        # 获得预测结果a:10*batch_size
        #testlabel:10*batch_size
        a=self.w_feedforward(testdata,activate,back=False)
        max1=np.argmax(a,axis=0)
        max2=np.argmax(testlabel,axis=0)
        accuracy=(np.sum((max1-max2) == 0))/(testlabel.shape[1])
        cost=np.sum(-(testlabel)*ln(a+pow(10,-20)))/testlabel.shape[1]
        return cost, accuracy
    
    
    def backprop(self,x,y,activate,dactivate,back=True):
        #x:输入：784*batch_size
        #y:输入标签：10*batch_size
        tri=[]
        nabla_mess = [np.zeros(mess.shape) for mess in self.messes]
        out,zm=self.w_feedforward(x,activate,back=True)
        for l in range(1, (self.num_layers)):
            if l==1:
                tri_=(out[-1]-y)
                tri.append(tri_)
            else:
                tri_=(np.dot(self.messes[-l+1].T, tri[-1]) * dactivate(zm[-l]))
                tri.append(tri_)
            nabla_mess[-l]=(np.dot(tri_,out[-l-1].T))/(np.shape(x)[1])
        return nabla_mess
    
    
    
    def adam_update(self,lr,mini_batch_size,activate,dactivate):
        data_x=train_data*1
        label_x=train_label*1
        data,label = mini_batch_generate(mini_batch_size,data_x,label_x)
        for j in range(data.shape[0]):
            delta_nabla_m = self.backprop(data[j],label[j],activate,dactivate,back=True)
            self.messes= self.Adam_theta.New_theta(self.messes,delta_nabla_m,lr)
            print('\r'+str(j)+'/'+str(int(data.shape[0])),end='')


           
        
        
    def SGD(self,mini_batch_size,epoch,lr0):
        evaluation_cost, evaluation_error = [], []
        training_cost, training_accuracy = [], []
        learning_rate=[]
        test1,label1=test_data,test_label
        for i in range(epoch):
            lr = divi_(lr0,i,30)
            print ("Epoch %s training complete" % i)
            self.adam_update(lr,mini_batch_size,relu,drelu)
            cost1,accuracy1 = self.evaluate(test1,label1,relu)
            evaluation_cost.append(cost1)
            evaluation_error.append((accuracy1))
            cost2,accuracy2 = self.evaluate(train_data,train_label,relu)
            training_cost.append(cost2)
            training_accuracy.append(accuracy2)
            print("the training Accuracy is:{} %".format((accuracy2)*100))
            print("the training cost is ",cost2)
            print("the Test accuracy is:{} %".format((accuracy1)*100))
            print("the cost is ",cost1)
        return evaluation_error

In [16]:
if __name__ == '__main__':
    N_err=[]
    for N in [100]:
        error = []
        for i in range(5):
            print("time is",i)
            net1=BPNetwork([784,N,N,10])
            test_error=net1.SGD(500,200,0.003)
            error.append(test_error*1)
            print(error[-1][-1])
            model_save('4hidn=100+N='+str(N)+'BP'+str(i+1)+'.pickle').model_s(net1)
        N_err.append(error*1)
#     np.save('fig4/scale/BP/4layer-BP+MINST_tail',N_err, allow_pickle=True)

time is 0
Epoch 0 training complete
119/120the training Accuracy is:94.62666666666667 %
the training cost is  0.1817847053551169
the Test accuracy is:94.57 %
the cost is  0.18388833821844056
Epoch 1 training complete
119/120the training Accuracy is:96.55833333333334 %
the training cost is  0.11646442768406834
the Test accuracy is:96.06 %
the cost is  0.12808369781132098
Epoch 2 training complete
119/120the training Accuracy is:97.38666666666667 %
the training cost is  0.08638752185886636
the Test accuracy is:96.6 %
the cost is  0.10838315392983655
Epoch 3 training complete
119/120the training Accuracy is:98.06 %
the training cost is  0.06640156980891068
the Test accuracy is:96.85000000000001 %
the cost is  0.10006290873975432
Epoch 4 training complete
119/120the training Accuracy is:98.115 %
the training cost is  0.059374533299129374
the Test accuracy is:97.04 %
the cost is  0.10171137307283379
Epoch 5 training complete
119/120the training Accuracy is:98.57333333333334 %
the training c

KeyboardInterrupt: 